In [1]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/JackyP/testing/master/datasets/nycflights.csv", index_col=0)
df["data"] = pd.to_datetime(df[['year', 'month', 'day']]) 
df.head()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute,data
1,2013,1,1,517.0,2.0,830.0,11.0,UA,N14228,1545,EWR,IAH,227.0,1400,5.0,17.0,2013-01-01
2,2013,1,1,533.0,4.0,850.0,20.0,UA,N24211,1714,LGA,IAH,227.0,1416,5.0,33.0,2013-01-01
3,2013,1,1,542.0,2.0,923.0,33.0,AA,N619AA,1141,JFK,MIA,160.0,1089,5.0,42.0,2013-01-01
4,2013,1,1,544.0,-1.0,1004.0,-18.0,B6,N804JB,725,JFK,BQN,183.0,1576,5.0,44.0,2013-01-01
5,2013,1,1,554.0,-6.0,812.0,-25.0,DL,N668DN,461,LGA,ATL,116.0,762,5.0,54.0,2013-01-01


In [2]:
df.columns

Index(['year', 'month', 'day', 'dep_time', 'dep_delay', 'arr_time',
       'arr_delay', 'carrier', 'tailnum', 'flight', 'origin', 'dest',
       'air_time', 'distance', 'hour', 'minute', 'data'],
      dtype='object')

In [3]:
usecols = ['dep_time','arr_time','carrier', 'tailnum', 'flight','air_time', 'distance','data']

df_raw = df.loc[(~df['dep_time'].isna()) \
                &(~df['dep_delay'].isna()) \
                &(~df['arr_time'].isna()) \
                &(~df['flight'].isna())] \
                    .loc[:,usecols]
                
                
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 328063 entries, 1 to 336770
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   dep_time  328063 non-null  float64       
 1   arr_time  328063 non-null  float64       
 2   carrier   328063 non-null  object        
 3   tailnum   328063 non-null  object        
 4   flight    328063 non-null  int64         
 5   air_time  327346 non-null  float64       
 6   distance  328063 non-null  int64         
 7   data      328063 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(2), object(2)
memory usage: 22.5+ MB


In [4]:
df['air_time'] = df['air_time'].fillna(0)

### Renomeando COLS

In [5]:
#colunas antigas
usecols = df_raw.columns

#colunas novas
new_cols = ['datetime_partida','datetime_chegada','companhia','id_aeronave','id_voo','tempo_voo','distancia','data_voo']

#list comprehension
columns_map = {usecols[i] : new_cols[i] for i in range(len(usecols))}
columns_map

{'dep_time': 'datetime_partida',
 'arr_time': 'datetime_chegada',
 'carrier': 'companhia',
 'tailnum': 'id_aeronave',
 'flight': 'id_voo',
 'air_time': 'tempo_voo',
 'distance': 'distancia',
 'data': 'data_voo'}

In [6]:
#aplicando o rename
df_raw.rename(columns=columns_map,inplace=True)

### TIPOS

In [7]:
df_raw['tempo_voo'] = df_raw['tempo_voo'].astype('float')
df_raw['data_voo'] = pd.to_datetime(df_raw['data_voo']) 
df_raw.dtypes

datetime_partida           float64
datetime_chegada           float64
companhia                   object
id_aeronave                 object
id_voo                       int64
tempo_voo                  float64
distancia                    int64
data_voo            datetime64[ns]
dtype: object

In [26]:
df_raw['datetime_partida'].apply(lambda x: len(x)).unique()

TypeError: object of type 'int' has no len()

### Criar novas colunas
#### Dado Tratado

In [9]:
df_raw.describe()

,datetime_partida,datetime_chegada,id_voo,tempo_voo,distancia,data_voo
count,328063.000000,328063.000000,328063.000000,327346.000000,328063.000000,328063
mean,1348.857710,1502.054999,1943.297534,150.686460,1048.826579,2013-07-03 03:59:26.772236032
min,1.000000,1.000000,1.000000,20.000000,80.000000,2013-01-01 00:00:00
25%,907.000000,1104.000000,544.000000,82.000000,509.000000,2013-04-05 00:00:00
50%,1400.000000,1535.000000,1468.000000,129.000000,888.000000,2013-07-04 00:00:00
75%,1744.000000,1940.000000,3413.000000,192.000000,1391.000000,2013-10-01 00:00:00
max,2400.000000,2400.000000,8500.000000,695.000000,4983.000000,2013-12-31 00:00:00
std,488.266036,533.264132,1621.525354,93.688305,735.959160,NaN


In [14]:
# df_raw.loc[df_raw['datetime_partida'] > 100].sort_values(by=['tempo_voo'],ascending=True)

,datetime_partida,datetime_chegada,companhia,id_aeronave,id_voo,tempo_voo,distancia,data_voo
176605,537.0,622.0,EV,N12167,4631,20.0,116,2013-04-13
13525,1355.0,1442.0,EV,N16911,4368,20.0,116,2013-01-16
310953,722.0,804.0,EV,N14920,6062,21.0,116,2013-09-03
233353,2338.0,17.0,EV,N11109,4276,21.0,116,2013-06-12
152532,1456.0,1533.0,EV,N12160,4118,21.0,116,2013-03-18
...,...,...,...,...,...,...,...,...
334198,555.0,953.0,EV,N133EV,5068,NaN,488,2013-09-28
334355,847.0,1130.0,EV,N14542,4510,NaN,725,2013-09-28
334413,1010.0,1344.0,EV,N12175,4412,NaN,1017,2013-09-28
334496,1214.0,1801.0,AA,N488AA,300,NaN,1372,2013-09-28


In [15]:
df_dw['tempo_voo_esperado'] = (df_raw['datetime_partida'] - df_raw['datetime_chegada'])/pd.Timedelta(hours= 1)

UFuncTypeError: ufunc 'divide' cannot use operands with types dtype('float64') and dtype('<m8[ns]')

In [18]:
len(df.loc[df['tailnum'].isna()])

2512

### Logging

In [19]:
import datetime
import logging

logging.basicConfig(filename='logging_flights.txt',level=logging.INFO)
logger = logging.getLogger()

In [22]:
logger.info(f'Inicio da execucao : {datetime.datetime.now())}')